<h3 align="center">
<font color='blue'>
Follows tries in  read-amazon-data-tries-and-sqlite.ipynb , but here for real anr in MySQL
</font>

<font color='putple'>
Tutorial on MySQL in python is @ http://zetcode.com/db/mysqlpython/
</font>
</h3> 

<h1 align="center">
<font color='magenta'>
make database of amazon metadata -- FILTERING BY SUBJECT -- 2runs 
</font>
</h1> 

In [ ]:
import MySQLdb as mdb
import sys
import nltk

In [ ]:
topics_mo = set([
 'dynamics',
 'electricity',
 'electromagnetic',
 'fourier',
 'hamiltonans',
 'hamiltonian',
 'lagrangian',
 'lagrangians',
 'laser',
 'lasers',
 "maxwell's",
 'mechanics',
 'nonrelativistic',
 'optical',
 'optics',
 'physics',
 'quantum',
 'radiation',
 'relativistic',
 'schroedinger',
 'spectra',
 'spectroscopic',
 'spectroscopy'
])

topics_compact = set([
 'hamiltonans',
 'hamiltonian',
 'lagrangian',
 'lagrangians',
 'mechanics',
 'nonrelativistic',
 'relativistic',
])

In [ ]:
def titleFits_byTitle(title,topic_set):
    tokens = nltk.word_tokenize(title.lower())
    return len(topic_set.intersection(tokens)) > 0

def titleFits_byAsin(asin,relevant_asin_set):
    return (asin in relevant_asin_set)

In [ ]:
con_f = mdb.connect('localhost', 'insightUser', 'insight15', 'amazon_filtered_compact');
#con_f = mdb.connect('localhost', 'insightUser', 'insight15', 'amazon_metadata_filtered');

In [ ]:
cur_f = con_f.cursor()
cur_f

In [ ]:
#--DEBUG--
con_f.close();

In [ ]:
cur_f

In [ ]:
cur_f.execute("SELECT VERSION()")
ver = cur_f.fetchone()
print "datavase version", ver

In [ ]:
cur_f.execute("DROP TABLE IF EXISTS metadata")
    
cur_f.execute("CREATE TABLE metadata(\
Id INT PRIMARY KEY AUTO_INCREMENT, \
asin VARCHAR(10), \
title VARCHAR(300),\
imURL VARCHAR(200),\
relevant_asins VARCHAR(550),\
salesRank INT)")

In [ ]:
cur_f.execute("SELECT DATABASE()")
print cur_f.fetchall()

cur_f.execute("SHOW TABLES")
print cur_f.fetchall()


In [ ]:
defSalesRank = -1
defUrl = 'https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcQVFwukzczimNzMwQwt9lRzCtarHKPiDffhfrx1Wm09QZjPFUOg'

In [ ]:
# INSERTING ALTOGETHER while filtering PASS 1

i=-1
i_failed = 0
i_passed = 0
PASS = 1

all_rel_set = set([])
gathered_set = set([])

with open('meta_Books.json','r') as f:
    for line in f:
        
        i+=1
        if (i%10000 ==0): print "line ", i
        
        #DEBUG
        #if (i>100): break
                        
        # PASS 1           
        try:
            line_dict = eval(line)
        
            #CHOOSE APPROPIATRE titleFits FOR PASS=1 OR PASS=2
            if not titleFits_byTitle(line_dict['title'],topics_compact): continue
            
            url_str = line_dict['imUrl']  if 'imUrl' in line_dict else defUrl
            salesRank = int(line_dict['salesRank']['Books'])  if 'salesRank' in line_dict else defSalesRank

            # find a better way to replace a single quote
            title_str = line_dict['title'].replace("'","").replace("&amp","")
            
            all_rel_set.add(line_dict['asin'])
            gathered_set.add(line_dict['asin'])
        
            # figure other relevant books
            rel_str = ''
            
            #if len(rel_str) > 500: break
            
            if 'related' in line_dict:
                for key in line_dict['related']:
                    vals = []
                    for val in line_dict['related'][key]:
                        rel_str += val
                        rel_str += ' '
                        all_rel_set.add(val) 

            all_insert =  "INSERT INTO metadata(asin,title,imUrl,relevant_asins,salesrank) \
                            VALUES('%s','%s','%s','%s','%d')"\
                            % (line_dict['asin'],\
                               title_str,\
                               url_str,\
                               rel_str,\
                               salesRank)

            cur_f.execute(all_insert)
            con_f.commit()
            i_passed +=1
            
            if (i_passed%10 ==0): print "passed: ", i_passed
            

        except Exception as e:
            i_failed += 1
            if (i_failed%100 ==0): print "    failed ", i_failed, "  out of", i
            # print "cant parse line ", i
            # print line
            print i, " -- ", e
            print
        
        if (i_passed > 2000): break
            
            #if i_failed > 2: break
            


In [ ]:
# INSERTING ALTOGETHER while filtering PASS 2

i=-1
i_failed = 0
i_passed = 0
PASS = 2

gathered_set = set([])

with open('meta_Books.json','r') as f:
    for line in f:
        
        i+=1
        if (i%10000 ==0): print "line ", i
        
        #DEBUG
        #if (i>100): break
                        
            
        try:
            line_dict = eval(line)
            
            asin = line_dict['asin']
        
            #CHOOSE APPROPIATRE titleFits FOR PASS=1 OR PASS=2
            #if not titleFits_byTitle(line_dict['title'],topics_compact): continue
            if not titleFits_byAsin(asin,all_rel_set_copy): continue
            
            url_str = line_dict['imUrl']  if 'imUrl' in line_dict else defUrl
            salesRank = int(line_dict['salesRank']['Books'])  if 'salesRank' in line_dict else defSalesRank

            # find a better way to replace a single quote
            title_str = line_dict['title'].replace("'","").replace("&amp","")
            
            #all_rel_set.add(line_dict['asin'])
            gathered_set.add(asin)
        
            # figure other relevant books
            rel_str = ''
            
            #if len(rel_str) > 500: break
            
            if 'related' in line_dict:
                for key in line_dict['related']:
                    vals = []
                    for val in line_dict['related'][key]:
                        rel_str += val
                        rel_str += ' '
                        #all_rel_set.add(val) 

            all_insert =  "INSERT INTO metadata(asin,title,imUrl,relevant_asins,salesrank) \
                            VALUES('%s','%s','%s','%s','%d')"\
                            % (asin,\
                               title_str,\
                               url_str,\
                               rel_str,\
                               salesRank)

            cur_f.execute(all_insert)
            con_f.commit()
            i_passed +=1
            
            if (i_passed%10 ==0): print "passed: ", i_passed
            

        except Exception as e:
            i_failed += 1
            if (i_failed%100 ==0): print "    failed ", i_failed, "  out of", i
            # print "cant parse line ", i
            # print line
            print i, " -- ", e
            print
        
        #if (i_passed > 2000): break
            
            #if i_failed > 2: break
            


In [ ]:
con_f.close()

In [ ]:


print i

print i_passed

print i_failed

print len(all_rel_set_copy)

print len(gathered_set)

In [ ]:
cur_f.execute("SELECT asin FROM metadata")


asins = []
for asin in cur_f.fetchall():
    asins.append(asin[0])

In [ ]:
len(asins)

In [ ]:
#pickle after the first pass
import pickle

#with open('relevant_asins_1pass.pkl','w') as f:
#    pickle.dump(all_rel_set,f)
    
with open('gathered_asins_2pass.pkl','w') as f:
    pickle.dump(asins,f)

<h1 align="center">
<font color='red'>
make database of amazon metadata -- FILTERING BY SUBJECT -- tries
</font>
</h1> 

In [ ]:
import MySQLdb as mdb
import sys

In [ ]:
topics_mo = set([
 'dynamics',
 'electricity',
 'electromagnetic',
 'fourier',
 'hamiltonans',
 'hamiltonian',
 'lagrangian',
 'lagrangians',
 'laser',
 'lasers',
 "maxwell's",
 'mechanics',
 'nonrelativistic',
 'optical',
 'optics',
 'physics',
 'quantum',
 'radiation',
 'relativistic',
 'schroedinger',
 'spectra',
 'spectroscopic',
 'spectroscopy'
])

topics_compact = set([
 'dynamics',
 'hamiltonans',
 'hamiltonian',
 'lagrangian',
 'lagrangians',
 'mechanics',
 'nonrelativistic',
 'relativistic',
])

In [ ]:
topics_amo

In [ ]:
import nltk

def titleFits(title,topic_set):
    tokens = nltk.word_tokenize(title.lower())
    return len(topic_set.intersection(tokens)) > 0

In [ ]:
con_f.close()

In [ ]:
con_f = mdb.connect('localhost', 'insightUser', 'insight15', 'amazon_metadata_filtered');

In [ ]:
con_f.close()

In [ ]:
cur_f = con_f.cursor()
cur_f

In [ ]:
cur_f.execute("SELECT VERSION()")
ver = cur_f.fetchone()
print "datavase version", ver

In [ ]:
cur_f.execute("DROP TABLE IF EXISTS amazon_metadata_FC")
    
cur_f.execute("CREATE TABLE amazon_metadata_FC(\
Id INT PRIMARY KEY AUTO_INCREMENT, \
asin VARCHAR(10), \
title VARCHAR(300),\
imURL VARCHAR(200),\
relevant_asins VARCHAR(550),\
salesRank INT)")

In [ ]:
defSalesRank = -1
defUrl = 'https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcQVFwukzczimNzMwQwt9lRzCtarHKPiDffhfrx1Wm09QZjPFUOg'

In [ ]:
import nltk

def titleFits(title,topic_set):
    tokens = nltk.word_tokenize(title.lower())
    return len(topic_set.intersection(tokens)) > 0

In [ ]:
# INSERTING ALTOGETHER while filtering

i=-1
i_failed = 0
i_passed = 0

all_rel_set = set([])

with open('meta_Books.json','r') as f:
    for line in f:
        
        i+=1
        if (i%10000 ==0): print "line ", i
            
        try:
            line_dict = eval(line)
        
            #UNCOMMENT THIS BEFORE THE GRAND RUN
            if not titleFits(line_dict['title'],topics_mo): continue
            
            url_str = line_dict['imUrl']  if 'imUrl' in line_dict else defUrl
            salesRank = int(line_dict['salesRank']['Books'])  if 'salesRank' in line_dict else defSalesRank

            # find a better way to replace a single quote
            title_str = line_dict['title'].replace("'","").replace("&amp","_amprsnd_")
            
            all_rel_set.add(line_dict['asin'])
        
            # figure other relevant books
            rel_str = ''
            
            #if len(rel_str) > 500: break
            
            if 'related' in line_dict:
                for key in line_dict['related']:
                    vals = []
                    for val in line_dict['related'][key]:
                        rel_str += val
                        rel_str += ' '
                        all_rel_set.add(val) 

            all_insert =  "INSERT INTO amazon_metadata_filtered(asin,title,imUrl,relevant_asins,salesrank) \
                            VALUES('%s','%s','%s','%s','%d')"\
                            % (line_dict['asin'],\
                               title_str,\
                               url_str,\
                               rel_str,\
                               salesRank)

            cur_f.execute(all_insert)
            con_f.commit()
            i_passed +=1
            if (i_passed%100 ==0): print "passed: ", i_passed
            

        except Exception as e:
            i_failed += 1
            if (i_failed%100 ==0): print "    failed ", i_failed, "  out of", i
            print "cant parse line ", i
            print line
            print e
            print
            
            #if i_failed > 2: break
            


In [ ]:
con_f.close()

In [ ]:
line_dict['title']

s = "The rogue of publishers' row;: Confessions of a publisher (A Banner Book)"

In [ ]:
cur_f.execute("SELECT * FROM amazon_metadata_filtered")
print cur_f.fetchall()

In [ ]:
con_f.close()

In [11]:
import pickle

with open('relevant_asins.pkl','w') as f:
    pickle.dump(all_rel_set,f)

NameError: name 'all_rel_set' is not defined

In [ ]:
!ls
import pickle
with open('relevant_asins_1pass.pkl','rb') as f:
    all_rel_set_copy = pickle.load(f)
    
len(all_rel_set_copy)

<h1 align="center">
<font color='red'>
--------- make database of amazon REVIEWS ----- FILTERING BY SUBJECT -- tries
</font>
</h1> 

In [ ]:
import MySQLdb as mdb
import sys

In [ ]:
con_f = mdb.connect('localhost', 'insightUser', 'insight15', 'amazon_reviews_filtered');
cur_f = con_f.cursor()

In [ ]:
cur_f.execute("SELECT VERSION()")
ver = cur_f.fetchone()
print "database version", ver

In [ ]:
cur_f.execute("DROP TABLE IF EXISTS amazon_reviews_filtered")
    
cur_f.execute("CREATE TABLE amazon_reviews_filtered(\
Id INT PRIMARY KEY AUTO_INCREMENT, \
asin VARCHAR(10), \
helpful1 INT,\
helpful2 INT,\
overall FLOAT, \
reviewText VARCHAR(10000),\
reviewerID VARCHAR(18),\
reviewSummary VARCHAR(500))")

In [ ]:
# INSERTING one while filtering

i=-1
i_failed = 0
i_passed = 0
RUN = 1

all_rel_set = set([])

with open('reviews_Books_100lines.json','r') as f:
    for line in f:
        
        i+=1
        if (i%40 ==0): print "line ", i
            
        try:
            line_dict = eval(line)
        
            #UNCOMMENT THIS BEFORE THE GRAND RUN
            #if not line_dict['asin'] in all_rel_set: continue
                
            review_str = line_dict['reviewText'].replace("'","")
            summary_str = line_dict['summary'].replace("'","") if 'summary' in line_dict else ''

            all_insert_r =  "INSERT INTO amazon_reviews_filtered\
            (asin, \
            helpful1, \
            helpful2,\
            overall,\
            reviewText,\
            reviewerID,\
            reviewSummary\
            ) \
            VALUES('%s','%d','%d','%f','%s','%s','%s')" % \
            (line_dict['asin'],\
             line_dict['helpful'][0],\
             line_dict['helpful'][1],\
             line_dict['overall'],\
             review_str,\
             line_dict['reviewerID'],\
             summary_str
           )

            cur_f.execute(all_insert_r)
            con_f.commit()
            i_passed +=1
            

        except Exception as e:
            i_failed += 1
            if (i_failed%100 ==0): print "    failed ", i_failed, "  out of", i
            print "cant parse line ", i
            print line
            print e
            print
            
            
            #if i_failed > 2: break
            



In [ ]:
con_f.close()

In [ ]:
print line_dict['asin']
print line_dict['helpful'][0]
print line_dict['helpful'][1]
print line_dict['overall']
print review_str
print line_dict['reviewerID']
print line_dict['summary']
print line_dict['reviewTime']
print line_dict['unixReviewTime']

In [ ]:
line_dict

<h1 align="center">
<font color='magenta'>
make database of amazon REVIEWS -- filtering by books -- 2runs 
</font>
</h1> 

In [1]:
import MySQLdb as mdb
import sys

In [3]:
!ls
import pickle
with open('gathered_asins_2pass.pkl','rb') as f:
    rel_asins = pickle.load(f)
    
len(rel_asins)

amazon_metadata_100lines.db	  read-amazon-data-MySQL.ipynb
amazon_metadata_phys.db		  read-amazon-data-tries-and-sqlite.ipynb
db_to_pandas_reviews_tries.ipynb  relevant_asins_1pass.pkl
gathered_asins_1pass.pkl	  relevant_asins.pkl
gathered_asins_2pass.pkl	  reviews_Books_100lines.json
meta_Books_100lines.json	  reviews_Books.json
meta_Books.json			  trycurl.json
ol_dump_200lines.txt		  Z-INSIGHT-DATA.lnk
ol_dump_2015-08-31.txt


10010

In [ ]:
def titleFits_byAsin(asin,relevant_asin_set):
    return (asin in relevant_asin_set)

In [4]:
con_f = mdb.connect('localhost', 'insightUser', 'insight15', 'amazon_filtered_compact');
#con_f = mdb.connect('localhost', 'insightUser', 'insight15', 'amazon_metadata_filtered');

cur_f = con_f.cursor()
cur_f

In [5]:
cur_f.execute("DROP TABLE IF EXISTS reviews")
    
cur_f.execute("CREATE TABLE reviews(\
Id INT PRIMARY KEY AUTO_INCREMENT, \
asin VARCHAR(10), \
helpful1 INT,\
helpful2 INT,\
overall FLOAT, \
reviewText VARCHAR(10000),\
reviewerID VARCHAR(18),\
reviewSummary VARCHAR(500))")

0L

In [6]:
cur_f.execute("SELECT DATABASE()")
print cur_f.fetchall()

cur_f.execute("SHOW TABLES")
print cur_f.fetchall()


(('amazon_filtered_compact',),)
(('metadata',), ('metadata_1pass',), ('reviews',), ('reviews_100lines',))


In [7]:
# GATHER BASED ON FILTERING BY ASINS

i=-1
i_failed = 0
i_passed = 0

gathered_asins = set([])

with open('reviews_Books.json','r') as f:
    for line in f:
        
        i+=1
        if (i%10000 ==0): print "line ", i
            
        try:
            line_dict = eval(line)
        
            #UNCOMMENT THIS BEFORE THE GRAND RUN
            if not line_dict['asin'] in rel_asins: continue
                
            review_str = line_dict['reviewText'].replace("'","")
            summary_str = line_dict['summary'].replace("'","") if 'summary' in line_dict else ''

            all_insert_r =  "INSERT INTO reviews\
            (asin, \
            helpful1, \
            helpful2,\
            overall,\
            reviewText,\
            reviewerID,\
            reviewSummary\
            ) \
            VALUES('%s','%d','%d','%f','%s','%s','%s')" % \
            (line_dict['asin'],\
             line_dict['helpful'][0],\
             line_dict['helpful'][1],\
             line_dict['overall'],\
             review_str,\
             line_dict['reviewerID'],\
             summary_str
           )

            cur_f.execute(all_insert_r)
            con_f.commit()
            i_passed +=1
            gathered_asins.add(line_dict['asin'])
            if (i_passed % 100 ==0): print ' passed ', i_passed
            

        except Exception as e:
            i_failed += 1
            if (i_failed%100 ==0): print "    failed ", i_failed, "  out of", i
            print "cant parse line ", i, e
            #print line
            #print e
            #print
            
            
            #if i_failed > 2: break
            



line  0
line  10000
line  20000
line  30000
line  40000
 passed  100
line  50000
line  60000
line  70000
line  80000
line  90000
line  100000
line  110000
line 

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:27: Warning: Data truncated for column 'reviewText' at row 1
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:27: Warning: Data truncated for column 'reviewerID' at row 1


 120000
 passed  200
line  130000
 passed  300
line  140000
 passed  400
 passed  500
 passed  600
line  150000
 passed  700
 passed  800
 passed  900
 passed  1000
line  160000
line  170000
line  180000
 passed  1100
line  190000
line  200000
line  210000
line  220000
line  230000
 passed  1200
line  240000
 passed  1300
 passed  1400
line  250000
line  260000
line  270000
line  280000
line  290000
 passed  1500
 passed  1600
 passed  1700
 passed  1800
 passed  1900
 passed  2000
 passed  2100
 passed  2200
line  300000
line  310000
 passed  2300
 passed  2400
 passed  2500
line  320000
line  330000
line  340000
 passed  2600
 passed  2700
 passed  2800
line  350000
 passed  2900
line  360000
line  370000
line  380000
line  390000
line  400000
line  410000
line  420000
 passed  3000
 passed  3100
line  430000
line  440000
 passed  3200
line  450000
 passed  3300
 passed  3400
 passed  3500
 passed  3600
line  460000
line  470000
line  480000
line  490000
line  500000
 passed  3700
 p

In [ ]:
con_f.close()

In [13]:
len(gathered_asins)

import pickle

with open('relevant_asins_reviews.pkl','w') as f:
    pickle.dump(gathered_asins,f)

In [14]:
print line_dict['asin']
print line_dict['helpful'][0]
print line_dict['helpful'][1]
print line_dict['overall']
print review_str
print line_dict['reviewerID']
print line_dict['summary']
print line_dict['reviewTime']
print line_dict['unixReviewTime']

0321503031
20
21
5.0
Its a good physics textbook. Physics is one of those subjects where you need examples to really understand how it works, and this book has a lot of them. Although the main text is pretty busy and the font tiny, the chapters are logically organized and usually contain a concise summary of their contents. Its a pretty good book.
A25PYXY8W8BCZS
Move up to creating more complex Flash
06 20, 2007
1182297600


In [ ]:
cur_f.execute("SELECT reviewText FROM reviews_100lines")

revs = cur_f.fetchall()

In [ ]:
revs

<h1 align="center">
<font color='red'>
TRIES
</font>
</h1> 

In [ ]:
!ls

In [ ]:
def collect_relevants(line_dict,all_rel_set):
    """ from a dictionary obtained from an amazon metadata line,
    creates the string entry for the 'related' field
    and updates the set of related books """

    if not('related' in line_dict):
        return all_rell_set, ""

    vals = []
    for val in relD[key]:
        vals.append(val)
        rel_str += val
        rel_str += ' '
        
    all_rel_set.update(vals)

In [ ]:
# INSERTING VALUE BY VALUE - TYOUT - not goes in production

i=-1
i_failed = 0
i_passed = 0

all_rel_set = set([])

with open('meta_Books_100lines.json','r') as f:
    for line in f:
        
        i+=1
        #if (i%10000 ==0): 
        print "line ", i
        
        line_dict = eval(line)
                
        asin_insert = "INSERT INTO amazon_metadata_filtered(asin) VALUE('%s')" \
                        % (line_dict['asin'])
        
        title_insert = "INSERT INTO amazon_metadata_filtered(title) VALUES('%s')" \
                        % (line_dict['title'])
            
        url_str = line_dict['imUrl']  if 'imUrl' in line_dict else defUrl
        
        url_insert = "INSERT INTO amazon_metadata_filtered(imURL) VALUES('%s')"\
                       % (url_str)
            update
        salesRank = line_dict['salesRank']['Books']  if 'salesRank' in line_dict else defSalesRank
        salesRank_insert = "INSERT INTO amazon_metadata_filtered(salesrank) VALUES('%d')" \
                       % (salesRank)
            
        cur_f.execute(asin_insert)
        cur_f.execute(title_insert)
        cur_f.execute(url_insert)
        cur_f.execute(salesRank_insert)
        
        # figure other relevant books
        
        rel_str = ''
        
        if 'related' in line_dict:
            
            for key in line_dict['related']:
                
                vals = []
                
                for val in line_dict['related'][key]:
                    vals.append(val)
                    rel_str += val
                    rel_str += ' '
        
                all_rel_set.update(vals) 
            
        rel_insert = "INSERT INTO amazon_metadata_filtered(relevant_asins) VALUES('%s')"\
                       % (rel_str)
            
        cur_f.execute(rel_insert)
            
        break

In [ ]:
# INSERTING ALTOGETHER while filtering

i=-1
i_failed = 0
i_passed = 0

all_rel_set = set([])

with open('reviews_Books_100lines.json','r') as f:
    for line in f:
        
        i+=1
        if (i%40 ==0): print "line ", i
            
        try:
            line_dict = eval(line)
        
            #UNCOMMENT THIS BEFORE THE GRAND RUN
            #if not line_dict['asin'] in all_rel_set: continue
                
            review_str = line_dict['reviewText'].replace("'","")
            summary_str = line_dict['summary'] if 'summary' in line_dict else ''

            
            all_insert_r =  "INSERT INTO amazon_reviews_filtered\
            (asin, \
            helpful1, \
            helpful2, \
            overall,\
            reviewText,\
            reviewerID,\
            reviewSummary\
            ) \
            VALUES('%s','%d','%d','%f',%s','%s','%s','%s',%d')" % \
            (line_dict['asin'],\
             line_dict['helpful'][0],\
             line_dict['helpful'][1],\
             line_dict['overall'],\
             review_str,\
             line_dict['reviewerID'],\
             summary_str)

            cur_f.execute(all_insert_r)
            con_f.commit()
            i_passed +=1
            

        except Exception as e:
            i_failed += 1
            if (i_failed%100 ==0): print "    failed ", i_failed, "  out of", i
            print "cant parse line ", i
            print all_insert_r
            print 
            print e
            print
            print
            
            break
            
            #if i_failed > 2: break
            



In [ ]:
tryset={'ff','reg'}

print ('ff' in tryset)